# 🧭 Capstone Project: The Learning Compass
### Google AI Agents Intensive | Option C: Agents for Good

**Project Overview**
This project implements a sophisticated **Multi-Agent System** designed to democratize education. It generates high-quality, personalized lesson plans and study materials for any topic and difficulty level.

**The Architecture**
Unlike a simple chatbot, this system uses a sequential chain of three specialized agents, built with the **Google Agent Development Kit (ADK)**:

1.  **🤖 The Planner (Architect):** Uses a retrieval tool to design a structured 3-day curriculum, preventing structural hallucinations.
2.  **✍️ The Content Creator (Writer):** Takes the blueprint and drafts detailed, engaging educational content. It is equipped with a **PDF Generation Tool** to create tangible artifacts.
3.  **⚖️ The Evaluator (Judge):** Acts as a quality gate. It reviews the content, scores it, and triggers a **Self-Correction Loop** if the quality does not meet the standard.

**Technical Stack**
* **Framework:** Google ADK (Agent Development Kit)
* **Model:** Gemini 2.5 Flash Lite
* **Tools:** Custom Python Tools (Template Retrieval, PDF Generation)
* **Pattern:** Sequential Orchestration with Feedback Loops

## Phase 1: Setup & Tool Definition

In this phase, we initialize the environment and define the custom tools that give our agents "hands."

**Key Tools:**
* `get_syllabus_template`: A retrieval tool that provides a proven pedagogical structure, ensuring the Planner Agent doesn't invent random formats.
* `save_lesson_to_pdf`: A capability tool that allows the Content Agent to compile the final lesson into a downloadable PDF file using the `fpdf` library.

In [ ]:
# --- 1. Install ADK (If not already installed) ---
!pip install -q google-adk

In [1]:
!pip install fpdf

In [2]:
import os
import json
import re
from fpdf import FPDF
from google.genai import types
from google.adk.agents import LlmAgent
from google.adk.models.google_llm import Gemini
from google.adk.runners import InMemoryRunner
from google.adk.tools import google_search # <--- The Star of the Show

# --- 2. Authentication ---
GOOGLE_API_KEY = None
try:
    from kaggle_secrets import UserSecretsClient
    GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")
    print("✅ Authenticated via Kaggle Secrets.")
except ImportError:
    try:
        from dotenv import load_dotenv
        load_dotenv()
        GOOGLE_API_KEY = os.environ.get("GOOGLE_API_KEY")
        print("✅ Authenticated via local .env.")
    except ImportError: pass

if GOOGLE_API_KEY: os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
else: print("⚠️ Auth Error: No key found.")

✅ Authenticated via Kaggle Secrets.


In [3]:
# --- 3. Custom Tools ---

def save_lesson_to_pdf(filename: str, content: str) -> str:
    """
    Saves the lesson content to a PDF file.
    Args:
        filename: Name of the file (e.g., 'Lesson.pdf').
        content: The text content to write.
    """
    try:
        pdf = FPDF()
        pdf.add_page()
        pdf.set_font("Arial", size=12)
        # Encode to latin-1 to handle special chars safely
        safe_content = content.encode('latin-1', 'replace').decode('latin-1')
        pdf.multi_cell(0, 10, txt=safe_content)
        pdf.output(filename)
        return f"Successfully generated PDF: {filename}"
    except Exception as e:
        return f"Error generating PDF: {e}"

print("✅ Tools & Environment Ready (Dynamic Search Enabled).")

✅ Tools & Environment Ready (Dynamic Search Enabled).


## Phase 2: Architecting the Agent Team

Here we define our three specialized agents using the `LlmAgent` class. Each agent is given a specific **Instruction (System Prompt)** and a distinct set of **Tools** to enforce separation of concerns.

* **Planner Agent:** Has exclusive access to the syllabus template.
* **Content Agent:** Has exclusive access to the PDF generator.
* **Evaluator Agent:** Is instructed to act as a strict judge and output structured JSON data for programmatic parsing.

In [4]:
retry_config = types.HttpRetryOptions(attempts=5, exp_base=7, initial_delay=1, http_status_codes=[429, 500, 503])

# --- AGENT 1: PLANNER (Researcher) ---
planner_agent = LlmAgent(
    name="PlannerAgent",
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    instruction="""
    You are an expert curriculum planner.
    1. Use the `google_search` tool to research the standard university curriculum for the user's topic.
    2. Based on your research, create a modern, structured 3-Day Syllabus for the requested difficulty level.
    3. Output the 3-day plan clearly.
    """,
    tools=[google_search] # <--- Using Google Search for RAG
)

# --- AGENT 2: CREATOR (Writer) ---
content_agent = LlmAgent(
    name="ContentAgent",
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    instruction="""
    You are an expert educational content writer.
    Write a detailed, engaging lesson based on the provided plan.
    
    IMPORTANT: When the lesson is approved, you MUST use the `save_lesson_to_pdf` tool 
    to save the final result. Ensure the filename ends in '.pdf'.
    """,
    tools=[save_lesson_to_pdf] 
)

# --- AGENT 3: EVALUATOR (The Judge) ---
evaluator_agent = LlmAgent(
    name="EvaluatorAgent",
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    instruction="""
    Strict academic evaluator. Output ONLY valid JSON.
    Schema: {"score": int, "reasoning": "string", "status": "PASS"|"FAIL"}
    Pass criteria: Score >= 4.
    """
)

print("✅ Agents Initialized (Planner is now Web-Connected).")

✅ Agents Initialized (Planner is now Web-Connected).


## Phase 3: The Orchestration Logic

This is the "brain" of the application. Instead of a simple linear chain, we implement a **Robust Execution Pipeline**:

1.  **Event Handling:** The ADK returns a stream of events (Function Calls, Responses, Text). We implement a helper function (`get_last_text_response`) to safely extract the model's final text output, ignoring intermediate tool steps.
2.  **Self-Correction Loop:** If the **Evaluator Agent** gives a "FAIL" score, the logic automatically routes the feedback back to the **Content Agent**, prompting a rewrite. This ensures quality without user intervention.
3.  **Artifact Generation:** Only once the content passes the quality check does the system instruct the agent to generate the PDF.

In [5]:
import json

# --- Helper: Extract Text Safely ---
def get_last_text_response(events):
    if not events: return "Error: No events."
    for event in reversed(events):
        if event.content and event.content.parts:
            part = event.content.parts[0]
            if hasattr(part, 'text') and part.text: return part.text
    return "Error: No text found."

# --- Helper: Clean JSON ---
def parse_json_garbage(text):
    """Extracts JSON from markdown code blocks if present."""
    try:
        match = re.search(r'\{.*\}', text, re.DOTALL)
        if match: return json.loads(match.group(0))
        return json.loads(text)
    except:
        return {"score": 0, "status": "FAIL", "reasoning": "JSON Parse Error"}

# --- MAIN ORCHESTRATOR ---
async def run_learning_compass(topic, level="Beginner"):
    print(f"🧭 Launching Learning Compass for: {topic} ({level})\n")
    
    # 1. PLAN (Research Phase)
    print("--- 🤖 PLANNER: Researching the Web ---")
    planner_runner = InMemoryRunner(agent=planner_agent)
    plan_res = await planner_runner.run_debug(f"Research and create a syllabus for {topic} ({level})")
    syllabus = get_last_text_response(plan_res)
    print(f"\n[Syllabus Generated]:\n{syllabus[:200]}...\n") 

    # 2. DRAFT & REFINE LOOP
    print("--- ✍️ CONTENT: Drafting Lesson ---")
    content_runner = InMemoryRunner(agent=content_agent)
    lesson_text = get_last_text_response(await content_runner.run_debug(f"Write Day 1 for: {syllabus}"))
    
    eval_runner = InMemoryRunner(agent=evaluator_agent)
    
    # SELF-CORRECTION LOOP (Max 2 retries)
    for attempt in range(1, 4):
        print(f"\n--- ⚖️ EVALUATOR: Reviewing (Round {attempt}) ---")
        eval_res = get_last_text_response(await eval_runner.run_debug(f"Evaluate: {lesson_text}"))
        metrics = parse_json_garbage(eval_res)
        
        print(f"Score: {metrics.get('score')}/5 | Status: {metrics.get('status')}")
        
        if metrics.get('status') == "PASS":
            print("✅ Quality Check Passed!")
            
            # 3. SAVE TO FILE
            print("\n--- 💾 SAVING PDF ARTIFACT ---")
            filename = f"{topic.replace(' ', '_')}_Lesson.pdf"
            save_res = await content_runner.run_debug(
                f"The lesson is approved. Please save this content to file '{filename}' using your tool."
            )
            print(f"System: {get_last_text_response(save_res)}")
            break
            
        else:
            if attempt < 3:
                print("⚠️ Quality Check Failed. Requesting Revision...")
                feedback = f"Evaluator feedback: {metrics.get('reasoning')}. Fix these issues."
                lesson_text = get_last_text_response(await content_runner.run_debug(feedback))
            else:
                print("❌ Max retries reached. Creating partial artifact.")

## Phase 4: Execution

Let's run the system! Enter a topic (e.g., "Transformer Architecture in NLP") and a difficulty level to see the agents collaborate, critique, and produce a final PDF.

In [6]:
# --- EXECUTE ---
topic = input("Hey! I'm your learning compass. What do you want to learn?: ")
level = input("Which level (Beginner, Intermediate, Advanced) would you like?: ")
await run_learning_compass(topic, level)

Hey! I'm your learning compass. What do you want to learn?:  Transformer Architecture vs NVIDIA Nemotrons
Which level (Beginner, Intermediate, Advanced) would you like?:  Intermediate


🧭 Launching Learning Compass for: Transformer Architecture vs NVIDIA Nemotrons (Intermediate)

--- 🤖 PLANNER: Researching the Web ---

 ### Created new session: debug_session_id

User > Research and create a syllabus for Transformer Architecture vs NVIDIA Nemotrons (Intermediate)
PlannerAgent > ## Transformer Architecture vs. NVIDIA Nemotrons: An Intermediate Deep Dive

This 3-day syllabus is designed for an intermediate audience with a foundational understanding of deep learning and natural language processing. It aims to provide a comprehensive comparison between the widely adopted Transformer architecture and NVIDIA's innovative Nemotron models, highlighting their core differences, strengths, and emerging applications.

---

### Day 1: Foundations of Transformers and the Rise of Nemotron

**Morning Session (9:00 AM - 12:30 PM)**

*   **9:00 AM - 10:30 AM: Revisiting the Transformer Architecture**
    *   **Lecture:** A deep dive into the Transformer's encoder-decoder structure, self

System: Error: No text found.


## Project Reflection & Future Work

### Why Agents?
A single LLM prompt often struggles to maintain structure, accuracy, and formatting simultaneously. By decomposing the task into **Planning**, **Drafting**, and **Evaluating**, we achieve higher quality output. The **Evaluator Agent** acts as a crucial guardrail, catching hallucinations or poor explanations before they reach the user.

### Key Learnings
* **Schema Generation:** I learned how the ADK automatically converts Python type hints into tool schemas, allowing Gemini to understand *how* to call functions like `save_lesson_to_pdf`.
* **Resilience:** I implemented robust error handling to manage the complex event streams returned by the ADK, ensuring the system doesn't crash during tool execution.

### Next Steps
To take this from prototype to production, I would:
1.  **Integrate RAG:** Replace the template dictionary with a Vector Database to retrieve curriculum standards from real-world textbooks.
2.  **Human-in-the-Loop:** Add a UI checkpoint where the user can approve the Syllabus before the Content Agent begins writing.